In [1]:
import pandas as pd
import numpy as np
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn
warnings.simplefilter('ignore')
import sys
sys.path.extend(['../src/'])
from config import *
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import *
mpl.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False
% matplotlib inline

config = Config()

### 数据读取

In [88]:
data = pd.read_excel('cache.xlsx')

In [89]:
data.columns

Index(['服务单号', '客户名称', '客户类型', '紧急程度', '问题发现日期', '分配工程师', '设备编号', '设备型号',
       '服务工时(小时)', '服务间隔天数', '上次维修时间', '装机日期', '维修服务内容', '问题汇总', '解决方案汇总',
       '装机年份'],
      dtype='object')

### 特征工程

由于每条样本都是一条单独的维修记录，不用把该仪器设备单独提出来做连续的时间线，只需要把每个时间点的仪器当成一台平行时空下的独立的仪器就好。

对于该条样本下，该仪器当前的状态构建以下特征：

1. 该仪器型号的平均使用年限：定义一台仪器淘汰所需要的平均年限，需要找工程师问有哪些仪器是已经报废的，报废时间及装机时间
2. 该仪器型号报废前的平均维修次数：可精确到各个部件各维修的次数
3. 该仪器最近一年的维修次数
4. 该仪器最近一年的维修次数占对应客户最近一年的总维修次数的比
5. 仪器对应客户最近一年的维修总次数
6. 仪器最近一年遭遇严重问题zp1的次数
7. 仪器最近一年遭遇zp1在总次数中的占比
8. 仪器距离上一次维修的时间间隔
9. 仪器距离上一次维护的时间间隔
10. 该仪器的已使用年限
11. 同类仪器对应使用年限时的平均维修次数
。。。

#### 在构造特征之前需要提取的信息

##### 每类型仪器各有多少台

In [90]:
data['设备型号'].value_counts()

1235-514                1683
GSL-120                  674
2021-0010                592
2081-0010                524
GSL-120/GSL-10           396
GCS3000DX2               393
1235-5220                391
1420-020                 389
TQD                      352
CDS-5                    308
1235-501                 240
Capillarys 2 FP          229
Xevo TQD                 126
Quattro Micro             82
6000                      77
KM1                       63
6000-0010                 61
CaptureStation            43
1420-012                  40
1296-026                  35
luminex 200               31
LX200                     24
00-0335                   23
GCS3000Dx2                23
UPLC                      20
KM2                       20
1296-0010                 19
1296-003                  12
1235-0110                  9
5014-0020                  4
CaptureStation(FISH)       3
TQS                        2
GSL-10                     2
MB4                        2
SD            

In [91]:
instrument_uniq = data.drop_duplicates(subset = '设备编号', keep = 'last')

In [92]:
instrument_uniq.loc[instrument_uniq['设备型号'].isnull(),:]

,服务单号,客户名称,客户类型,紧急程度,问题发现日期,分配工程师,设备编号,设备型号,服务工时(小时),服务间隔天数,上次维修时间,装机日期,维修服务内容,问题汇总,解决方案汇总,装机年份
3830,BS-SHFW-维修部201708290097,浙江博圣生物技术股份有限公司,一般客户,未知,2017-08-27,陈凌云,BS-2016-WTS-TQS-01,NaN,1.0,2.0,2017-08-27,2016-07-01,维修,基因小镇的氮气发生器亮service灯氮气发生器亮service等,1.氮气发生器型号为genius3020，软件版本为v3.0。genius3020有4个空气...,2016
4245,BS-SHFW-维修部201711133460,无锡市妇幼保健院,VIP,未知,2017-11-13,戴明路,BS-2017-NS550-01,NaN,NaN,NaN,2017-11-13,2017-11-15,装机/移机,协助厂家工程师安装一套安诺优达NIPT系统陪同装机,NaN,2017


发现共有两台仪器数据没有

In [93]:
data.loc[data['设备编号'] == 'BS-2016-WTS-TQS-01',:]

,服务单号,客户名称,客户类型,紧急程度,问题发现日期,分配工程师,设备编号,设备型号,服务工时(小时),服务间隔天数,上次维修时间,装机日期,维修服务内容,问题汇总,解决方案汇总,装机年份
1502,BS-SHFW-维修部201708290097,浙江博圣生物技术股份有限公司,一般客户,未知,2017-08-27,陈凌云,BS-2016-WTS-TQS-01,NaN,1.0,2.0,2016-07-01,2016-07-01,维修,基因小镇的氮气发生器亮service灯氮气发生器亮service等,1.氮气发生器型号为genius3020，软件版本为v3.0。genius3020有4个空气...,2016
3830,BS-SHFW-维修部201708290097,浙江博圣生物技术股份有限公司,一般客户,未知,2017-08-27,陈凌云,BS-2016-WTS-TQS-01,NaN,1.0,2.0,2017-08-27,2016-07-01,维修,基因小镇的氮气发生器亮service灯氮气发生器亮service等,1.氮气发生器型号为genius3020，软件版本为v3.0。genius3020有4个空气...,2016


In [94]:
data.loc[data['设备编号'].apply(lambda x: 'WTS' in x), ['设备编号','设备型号']]

,设备编号,设备型号
0,BS-2016-WTS-TQD-16,Xevo TQD
1,BS-2016-WTS-TQD-01,TQD
78,BS-2014-WTS-TQD-11,TQD
203,BS-2013-WTS-QM-01,Quattro Micro
240,BS-2014-WTS-TQD-08,TQD
248,BS-2015-WTS-TQD-06,TQD
312,OD-2016-WTS-QM-01,Quattro Micro
314,OD-2016-WTS-QM-01,Quattro Micro
319,BS-2014-WTS-TQD-05,TQD
376,BS-2014-WTS-TQD-05,TQD


怀疑是陈凌云填错了，其实是TQD不是TQS

In [95]:
data.loc[data['设备编号'] == 'BS-2016-WTS-TQS-01','设备编号'] = 'BS-2016-WTS-TQD-01'
data.loc[data['设备编号'] == 'BS-2016-WTS-TQD-01','设备型号'] = 'TQD'

In [96]:
data.loc[data['设备编号'].apply(lambda x: 'NS' in x), ['设备编号','设备型号']]

,设备编号,设备型号
1917,BS-2017-NS550-01,NaN
4245,BS-2017-NS550-01,NaN
6875,AN-NS550-2018001,Nextseq 550AR


In [97]:
data.loc[data['设备编号'] == 'BS-2017-NS550-01', '设备型号'] = 'Nextseq 550AR'

In [98]:
instrument_uniq = data.drop_duplicates(subset = '设备编号', keep = 'last')

In [99]:
instrument_uniq['设备型号'].value_counts()

1235-514                160
1420-020                148
GSL-120/GSL-10          142
2081-0010               104
CDS-5                    91
1235-5220                72
2021-0010                71
TQD                      68
GCS3000DX2               63
Capillarys 2 FP          40
CaptureStation           33
1235-501                 28
6000-0010                26
Xevo TQD                 25
luminex 200              20
Quattro Micro            19
1296-026                 11
KM1                       7
1296-003                  7
GCS3000Dx2                6
1420-012                  5
GSL-120                   5
KM2                       4
00-0335                   4
UPLC                      3
1296-0010                 3
6000                      2
LX200                     2
Nextseq 550AR             2
CaptureStation(FISH)      2
5014-0020                 2
GSL-10                    1
MB8/MB4                   1
BGI SEQ-500               1
SLAN-96S                  1
TQS                 

#### 文本挖掘，重要问题zq1

1. 定义一个函数可以分词出单仪器型号的所有实体名词
2. 圈出其中的问题部分定义为zq1保存在文件problem.txt中

In [208]:
import jieba.posseg as pseg
# jieba.load_userdict('../source/words/userdict.txt')
# 创建停用词list  
def stopwordslist(filepath):  
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]  
    return stopwords    
  
# 对句子进行分词  
def seg_sentence(sentence):  
    sentence_seged = jieba.cut(sentence.strip(), HMM=True, cut_all=False)  
#     sentence_seged = jieba.cut_for_search(sentence.strip(), HMM=True)
    stopwords = stopwordslist('../source/words/中文停用词表.txt')  # 这里加载停用词的路径  
    outstr = ''  
    for word in sentence_seged:  
        if word not in stopwords:  
            if word != '\t':  
                outstr += word  
                outstr += " "  
    return outstr

In [210]:
data['设备型号'].unique()

array(['Xevo TQD', 'TQD', 'GSL-120', '1296-026', '1235-514', '00-0335',
       '6000', '1420-020', 'CDS-5', '2081-0010', 'GCS3000DX2',
       '2021-0010', '1296-0010', '1235-501', '1235-5220', '1235-0110',
       'Capillarys 2 FP', '1296-003', 'KM1', '1420-012', 'Quattro Micro',
       'KM2', 'UPLC', 'LX200', nan, 'MB4', '5014-0020', 'GSL-10',
       'Nextseq 550AR', 'GSL-120/GSL-10', 'CaptureStation', '6000-0010',
       'luminex 200', '1296-071', 'SD', 'BGI SEQ-500', 'GCS3000Dx2',
       'TQS', 'MB8/MB4', 'CaptureStation(FISH)', 'SLAN-96S'], dtype=object)

In [223]:
data_1235 = data.loc[(data['设备型号'] == '1235-514')&(data['维修服务内容'] == '维修'),:]

In [235]:
import jieba
from jieba import analyse

cache = ""
for i in list(data_1235['问题汇总']):
    try:
        cache = cache + i
    except:
        cache = cache + ""
content = " ".join(jieba.cut(cache))
allowP = ('nb','n','nr', 'ns','nt','nz')
key_words = analyse.extract_tags(content, topK=2000, withWeight=True, allowPOS= allowP)
textrank = analyse.textrank(content,topK = 200,withWeight = True, allowPOS = allowP)
keywords = dict()
for i in key_words:
    keywords[i[0]] = i[1]

In [236]:
textrank

[('妇幼', 1.0),
 ('故障', 0.7492782442049589),
 ('仪器', 0.6682865671057684),
 ('问题', 0.5667823213161618),
 ('联机', 0.5345829804546672),
 ('过程', 0.47608818043876816),
 ('医院', 0.47445664239890006),
 ('无法', 0.45748260288070713),
 ('样本', 0.4545378604493695),
 ('错误', 0.435086805928448),
 ('注射器', 0.404199082816038),
 ('标准', 0.395802668452305),
 ('洗板', 0.36603051102809586),
 ('卡住', 0.36377898907311973),
 ('右枪', 0.3563651449575343),
 ('试剂', 0.33412564183562926),
 ('真空', 0.29750051360500696),
 ('品仓', 0.2548821087030769),
 ('压力', 0.25451862634859107),
 ('开机', 0.2447474568985956),
 ('产筛', 0.2346459647212176),
 ('客户', 0.2262260446041884),
 ('气泡', 0.22611866133033154),
 ('上门', 0.22186348590820448),
 ('时报', 0.21718531861408785),
 ('妇幼保健', 0.20006621292277538),
 ('异响', 0.19931673058292124),
 ('电脑', 0.1953574948046611),
 ('皮带', 0.183349957643112),
 ('真空泵', 0.1747381738523576),
 ('电梯', 0.17042906073156971),
 ('条码', 0.14814580899963972),
 ('结果', 0.14713887093052375),
 ('报警', 0.145898728003628),
 ('系统', 0.1457

In [233]:
pd.Series(keywords)

妇幼      0.350948
故障      0.327665
注射器     0.256860
仪器      0.228327
联机      0.225487
右枪      0.193551
样本      0.187043
卡住      0.174309
洗板      0.162487
过程      0.151140
上门      0.138006
试剂      0.134511
产筛      0.126645
气泡      0.122237
错误      0.114193
问题      0.107743
无法      0.107438
客户      0.107384
异响      0.100360
枪头      0.097974
皮带      0.086890
真空      0.085865
电梯      0.081438
品仓      0.078854
真空泵     0.077973
标准      0.077893
开机      0.073303
板头      0.071686
医院      0.064452
蓝屏      0.063972
工程师     0.060789
报警      0.058536
板机      0.058482
板框      0.058078
时报      0.055843
针报      0.052569
年度      0.051957
电脑      0.051924
妇幼保健    0.051706
校准      0.051642
性能      0.050420
结果      0.048438
条码      0.047460
温控      0.046365
帽子      0.046065
洗液      0.045316
弹簧      0.044877
妇儿      0.042523
风扇      0.042141
荧光      0.041315
压力      0.040926
血片      0.040622
液面      0.039629
软件      0.038202
码器      0.035843
季度      0.034716
历史记录    0.034375
退板      0.033453
滤光片     0.0333

#### 特征构造

由于把每一条记录都当做一台单独平行时空的仪器，所以需要将该样本维修时间之前的所有样本提出来当做原始数据  
最终要对每一条记录跑一个大循环来计算每一个特征

In [28]:
data.columns

Index(['服务单号', '客户名称', '客户类型', '紧急程度', '问题发现日期', '分配工程师', '设备编号', '设备型号',
       '服务工时(小时)', '服务间隔天数', '上次维修时间', '装机日期', '维修服务内容', '问题汇总', '解决方案汇总'],
      dtype='object')

In [100]:
data['问题发现日期'] = pd.to_datetime(data['问题发现日期'], errors = 'coerce')
data = data.sort_values(by = '问题发现日期')

##### 该仪器最近一年的维修次数

In [101]:
data = data.reset_index(drop = True)

def service_count_year(data):
    instrument_id = data.iloc[-1]['设备编号']
    cache_ins = data.loc[data['设备编号'] == instrument_id,:]    
    cache_ins['year'] = cache_ins['问题发现日期'].apply(lambda x: x.year)
    last_year = cache_ins.iloc[-1]['year']
    return cache_ins['year'].value_counts()[last_year]

In [102]:
from tqdm import tqdm_notebook

service_count = []
for ix in tqdm_notebook(data.index):
    data_time = data.iloc[:ix+1].copy()
    service_count.append(service_count_year(data_time))

HBox(children=(IntProgress(value=0, max=6904), HTML(value='')))

In [103]:
data['最近一年维修次数'] = service_count

##### 该客户最近一年的维修总次数

In [104]:
def client_service_count_year(data):
    client_id = data.iloc[-1]['客户名称']
    cache_ins = data.loc[data['客户名称'] == client_id,:]    
    cache_ins['year'] = cache_ins['问题发现日期'].apply(lambda x: x.year)
    last_year = cache_ins.iloc[-1]['year']
    return cache_ins['year'].value_counts()[last_year]

In [105]:
client_service_count = []
for ix in tqdm_notebook(data.index):
    data_time = data.iloc[:ix+1].copy()
    client_service_count.append(client_service_count_year(data_time))

HBox(children=(IntProgress(value=0, max=6904), HTML(value='')))

In [106]:
data['最近一年该客户的维修总次数'] = client_service_count

##### 该仪器最近一年的维修次数占对应客户最近一年的总维修次数的比

In [107]:
data['年修占比'] = data['最近一年维修次数']/data['最近一年该客户的维修总次数']

##### 仪器距离上一次维修维护的时间间隔

In [148]:
data.loc[pd.to_datetime(data['装机日期'])< pd.to_datetime('1999-05-01'), '装机日期'] = '2005-06-28'

In [149]:
data.loc[(data['装机日期'].isnull())&(data['装机年份'].notnull()),'装机日期'] = data.loc[(data['装机日期'].isnull())&(data['装机年份'].notnull()),'装机年份']
data.loc[data['装机日期'] == '20XX','装机日期'] = '2016'
data.loc[data['上次维修时间'].isnull(),'上次维修时间'] = data.loc[data['上次维修时间'].isnull(),'装机日期']
data.loc[data['上次维修时间'] == '20XX','上次维修时间'] = '2016'

##### 该仪器的已使用年限

In [150]:
days_cache = (pd.to_datetime(data['问题发现日期'], errors = 'coerce') - pd.to_datetime(data['装机日期'], errors = 'coerce')).apply(lambda x: x.days)
data['已使用年限'] = round(days_cache/365.25,2)

In [151]:
data.loc[data['已使用年限']<0,['设备编号','问题发现日期','装机日期','维修服务内容']]

,设备编号,问题发现日期,装机日期,维修服务内容
0,BS-2016-WTS-TQD-16,2016-06-30,2016-10-08 00:00:00,维修
105,BS-2016-TMN-CDS-10,2016-09-19,2016-09-22 00:00:00,维修
126,BS-2016-TMN-CDS-17,2016-09-26,2016-09-30 00:00:00,维修
136,BS-2016-SBA-CP2-10,2016-10-10,2016-10-13 00:00:00,维修
187,BS-2016-LAK-KM-03,2016-10-31,2016-11-02 00:00:00,维修
197,BS-2016-PKI-AD-08,2016-11-01,2016-12-15 00:00:00,维修
230,BS-2016-TMN-CDS-11,2016-11-11,2016-11-15 00:00:00,维修
264,BS-2016-SBA-CP2-09,2016-11-21,2016-11-25 00:00:00,维修
272,BS-2016-PKI-AD-10,2016-11-21,2016-11-23 00:00:00,维修
282,BS-2016-PKI-AD-08,2016-11-24,2016-12-15 00:00:00,维修


可以看到很多已使用年限信息<0的样本  
一大部分是装机/移机的样本，这部分是可以直接改为0的  
其他的一部分，一部分是填写错误

In [152]:
data.loc[data['已使用年限']<0, '已使用年限'] = 0

##### 同类仪器对应使用年限时的平均维修次数

由于是维修次数，所以需要把维修以外的样本剃了

In [172]:
data_repair = data.loc[data['维修服务内容'] == '维修', :]

In [183]:
data_repair['已使用年限_round'] = data_repair['已使用年限'].apply(lambda x: round(x))
instrument_count = instrument_uniq['设备型号'].value_counts().to_dict()
instrument_average_service = {}
for instrument in instrument_count.keys():
    instrument_average_service[instrument] = (data_repair.loc[data_repair['设备型号'] == instrument, '已使用年限_round'].value_counts()/instrument_count[instrument]).apply(lambda x: round(x, 3))
average_service_stats = pd.concat(instrument_average_service,axis = 1)
# average_service_stats.fillna('', inplace = True)

In [184]:
average_service_stats.loc['仪器台数'] = instrument_uniq['设备型号'].value_counts().to_dict()

In [185]:
pd.options.display.max_columns = 200
average_service_stats = average_service_stats.T.sort_values(by = '仪器台数', ascending = False).T

In [186]:
average_service_stats

,1235-514,1420-020,GSL-120/GSL-10,2081-0010,CDS-5,1235-5220,2021-0010,TQD,GCS3000DX2,Capillarys 2 FP,CaptureStation,1235-501,6000-0010,Xevo TQD,luminex 200,Quattro Micro,1296-026,1296-003,KM1,GCS3000Dx2,1420-012,GSL-120,KM2,00-0335,UPLC,1296-0010,LX200,Nextseq 550AR,6000,5014-0020,CaptureStation(FISH),GSL-10,BGI SEQ-500,MB4,MB8/MB4,SLAN-96S,TQS
0,0.119,0.014,0.070,0.327,0.352,1.375,1.239,0.044,0.524,0.425,0.030,NaN,NaN,0.76,0.15,0.421,NaN,NaN,2.714,NaN,0.2,9.2,0.5,NaN,2.667,NaN,2.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.319,0.054,0.169,0.558,0.220,1.097,1.873,0.794,0.825,0.675,0.121,0.250,0.500,2.08,0.25,0.368,0.364,0.143,0.571,0.167,0.4,17.2,0.5,2.75,1.333,1.000,1.0,NaN,4.5,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,0.781,0.074,0.317,0.644,0.044,0.278,1.507,1.000,0.714,0.450,0.061,0.071,0.231,0.28,0.15,0.211,0.818,0.571,NaN,0.167,NaN,17.2,0.5,2.25,2.000,2.333,3.0,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.031,0.014,0.218,0.567,0.132,0.069,0.408,0.868,0.794,0.700,0.030,1.607,0.115,0.04,0.10,0.053,0.182,NaN,0.429,1.000,NaN,14.0,0.5,0.75,0.333,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.144,0.020,0.148,0.346,0.220,NaN,0.127,0.603,0.540,0.650,NaN,1.679,NaN,NaN,0.15,0.368,0.727,NaN,0.571,0.500,NaN,9.4,NaN,NaN,NaN,NaN,0.5,NaN,6.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.988,0.014,0.099,0.106,0.011,NaN,NaN,0.382,0.222,0.350,NaN,0.571,0.115,NaN,0.20,0.526,0.364,NaN,NaN,NaN,NaN,8.6,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0.650,0.034,0.134,NaN,0.132,NaN,NaN,NaN,0.079,0.025,0.030,0.143,0.077,NaN,NaN,0.474,0.182,NaN,NaN,0.333,NaN,9.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,0.588,0.027,0.134,NaN,0.077,NaN,NaN,0.015,0.095,NaN,0.061,0.143,NaN,NaN,NaN,0.421,NaN,NaN,0.714,NaN,NaN,4.4,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0.331,0.027,0.049,NaN,0.022,NaN,NaN,NaN,0.111,NaN,0.030,NaN,NaN,NaN,NaN,NaN,0.182,NaN,NaN,NaN,NaN,3.2,NaN,NaN,NaN,0.667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0.338,0.054,0.028,NaN,0.055,NaN,NaN,NaN,0.048,NaN,0.061,NaN,NaN,NaN,NaN,0.105,NaN,0.143,0.286,0.167,NaN,1.2,NaN,NaN,NaN,0.667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


从目前得到的结果可以看出几点：
1. 1235在相同年限上的维修次数明显要高于1420
2. 平均年限维修次数呈现先增后降的趋势，这可能与到一定年限就优先换新报废而不在维修维护有关
3. 个别仪器的样本较多，部分仪器较少，需要合并统计才能达到效果
4. 对于过于少的仪器，这类特征不采用趋势映射而直接做中位数插补

为了这张表完整可查，这边先进行中位数插补

In [187]:
average_service_stats.drop('仪器台数', axis = 0, inplace = True)

for col in average_service_stats.columns:
    average_service_stats[col] = average_service_stats[col].fillna(average_service_stats[col].median())

average_service_stats.fillna(0, inplace = True)

In [196]:
average_service_map = average_service_stats.to_dict()

average_service_map2 = {}
for k1 in average_service_map.keys():
    for k2 in average_service_map[k1].keys():
        average_service_map2['{}_{}'.format(k1, k2)] = average_service_map[k1][k2]

In [199]:
data['设备型号_已使用年限'] = data['设备型号'] + '_' + data['已使用年限_round'].astype(int).astype(str)

In [205]:
data['设备年平均维修次数'] = pd.to_numeric(data['设备型号_已使用年限'].map(average_service_map2), errors = 'coerce')

In [206]:
data.drop(['设备型号_已使用年限', '已使用年限_round'], axis = 1, inplace = True)